In [4]:
from geopy.geocoders import Nominatim

def get_coordinates(location):
    geolocator = Nominatim(user_agent="MyGeocoderApp/1.0")

    location_info = geolocator.geocode(location)

    if location_info:
        return str(location_info.latitude), str(location_info.longitude)
    else:
        print("Location not found")
        return None, None

lat,lng = get_coordinates('Dhone')
print(lat,lng)

15.3969719 77.8718482


In [5]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('API_KEY')

url = "https://ai-weather-by-meteosource.p.rapidapi.com/hourly"

querystring = {"lat":"12.99","lon":"77.62","timezone":"auto","language":"en","units":"auto"}

headers = {
	"X-RapidAPI-Key": "238768ee71mshcfd2270700c8ea4p169ed6jsnf8a59efee43f",
	"X-RapidAPI-Host": "ai-weather-by-meteosource.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring).json()

pprint(response)

{'elevation': 903,
 'hourly': {'data': [{'cloud_cover': 98,
                      'date': '2025-04-03T22:00:00',
                      'dew_point': 17.2,
                      'feels_like': 24.8,
                      'humidity': 67,
                      'icon': 11,
                      'ozone': 263.46,
                      'precipitation': {'total': 0.8, 'type': 'rain'},
                      'pressure': 1015,
                      'probability': {'freeze': 0.0,
                                      'precipitation': 95,
                                      'storm': 24},
                      'summary': 'Rain',
                      'temperature': 23.5,
                      'uv_index': None,
                      'visibility': 12.66,
                      'weather': 'rain',
                      'wind': {'angle': 156,
                               'dir': 'SSE',
                               'gusts': 4.4,
                               'speed': 2.5},
                      'wind_c

In [6]:
time = []
temp = []
summary = []
wind_speed = []
cloud_cover = []
humidity = []

for i in range(len(response['hourly']['data'])):
    cur_time = response['hourly']['data'][i]['date']
    cur_temp = response['hourly']['data'][i]['temperature']
    speed = response['hourly']['data'][i]['wind']['speed']
    cloud = response['hourly']['data'][i]['cloud_cover']
    humid = response['hourly']['data'][i]['humidity']
    weather = response['hourly']['data'][i]['summary']

    time.append(cur_time)
    temp.append(cur_temp)
    wind_speed.append(speed)
    cloud_cover.append(cloud)
    humidity.append(humid)
    summary.append(weather)

In [7]:
import pandas as pd

data = {'time':time,
        'temp(C)':temp,
        'wind speed':wind_speed,
        'cloud cover':cloud_cover,
        'humidity':humidity,
        'summary':summary}

df = pd.DataFrame(data)

In [8]:
df.head()

,time,temp(C),wind speed,cloud cover,humidity,summary
0,2025-04-03T22:00:00,23.5,2.5,98.0,67,Rain
1,2025-04-03T23:00:00,23.2,1.8,93.0,70,Rain
2,2025-04-04T00:00:00,22.8,1.3,98.0,72,Rain
3,2025-04-04T01:00:00,22.5,1.2,100.0,72,Thunderstorm
4,2025-04-04T02:00:00,22.0,1.7,100.0,67,Overcast


In [9]:
from datetime import datetime

df = df[df['time'].str.find(datetime.now().date().isoformat())==0].head()

In [10]:
df

,time,temp(C),wind speed,cloud cover,humidity,summary
0,2025-04-03T22:00:00,23.5,2.5,98.0,67,Rain
1,2025-04-03T23:00:00,23.2,1.8,93.0,70,Rain


In [11]:
df.loc[:,'time'] = df.loc[:,'time'].replace(datetime.now().date().isoformat()+'T','',regex=True)

In [12]:
df

,time,temp(C),wind speed,cloud cover,humidity,summary
0,22:00:00,23.5,2.5,98.0,67,Rain
1,23:00:00,23.2,1.8,93.0,70,Rain


In [13]:
df.loc[:,'temp(F)'] = df.loc[:,'temp(C)'].apply(lambda x: 32+((9/5)*x))

In [14]:
df.loc[:,'temp(K)'] = df.loc[:,'temp(C)'].apply(lambda x: 273+x)